# Scapy 실습

In [1]:
from scapy.all import *

In [2]:
# Scapy에서 제공하는 기능들
lsc()

IPID_count            : Identify IP id values classes in a list of packets
arp_mitm              : ARP MitM: poison 2 target's ARP cache
arpcachepoison        : Poison targets' ARP cache
arping                : Send ARP who-has requests to determine which hosts are up
arpleak               : Exploit ARP leak flaws, like NetBSD-SA2017-002.
bind_layers           : Bind 2 layers on some specific fields' values.
bridge_and_sniff      : Forward traffic between interfaces if1 and if2, sniff and return
chexdump              : Build a per byte hexadecimal representation
computeNIGroupAddr    : Compute the NI group Address. Can take a FQDN as input parameter
corrupt_bits          : Flip a given percentage (at least one bit) or number of bits
corrupt_bytes         : Corrupt a given percentage (at least one byte) or number of bytes
defrag                : defrag(plist) -> ([not fragmented], [defragmented],
defragment            : defragment(plist) -> plist defragmented as much as possible 
dhcp_r

## 패킷 구조

> `ls(obj, case_sensitive, verbose)`

    - obj: 패킷 혹은 패킷 이름 
    - case_sensitive (bool): obj가 스트링일 경우 대소문자 구분 여부
    - verbose (bool): 추가적인 정보 로깅 여부

### IPv4 패킷 구조

<img src="https://www.dropbox.com/s/7dn0tzw9nzt2dcw/ip.png?raw=1" width="466" height="216"/>

In [3]:
ls(IP(), verbose=True)

version    : BitField  (4 bits)                  = 4               ('4')
ihl        : BitField  (4 bits)                  = None            ('None')
tos        : XByteField                          = 0               ('0')
len        : ShortField                          = None            ('None')
id         : ShortField                          = 1               ('1')
flags      : FlagsField                          = <Flag 0 ()>     ('<Flag 0 ()>')
               MF, DF, evil
frag       : BitField  (13 bits)                 = 0               ('0')
ttl        : ByteField                           = 64              ('64')
proto      : ByteEnumField                       = 0               ('0')
chksum     : XShortField                         = None            ('None')
src        : SourceIPField                       = '127.0.0.1'     ('None')
dst        : DestIPField                         = '127.0.0.1'     ('None')
options    : PacketListField                     = []              ('[

### TCP 패킷 구조

<img src="https://www.dropbox.com/s/wuo7g9vjzydto8s/tcp.png?raw=1" width="521" height="225"/>

In [4]:
ls(TCP(), verbose=True)

sport      : ShortEnumField                      = 20              ('20')
dport      : ShortEnumField                      = 80              ('80')
seq        : IntField                            = 0               ('0')
ack        : IntField                            = 0               ('0')
dataofs    : BitField  (4 bits)                  = None            ('None')
reserved   : BitField  (3 bits)                  = 0               ('0')
flags      : FlagsField                          = <Flag 2 (S)>    ('<Flag 2 (S)>')
               F, S, R, P, A, U, E, C, N
window     : ShortField                          = 8192            ('8192')
chksum     : XShortField                         = None            ('None')
urgptr     : ShortField                          = 0               ('0')
options    : TCPOptionsField                     = []              ("b''")


- `/` 연산자로 패킷을 쌓을 수 있음
    - Data Link Layer / Network Layer / Transport Layer


In [5]:
packet = Ether() / IP() / TCP()
print(packet.summary())
print(packet[IP].summary())

Ether / IP / TCP 127.0.0.1:ftp_data > 127.0.0.1:http S
IP / TCP 127.0.0.1:ftp_data > 127.0.0.1:http S


- 각 필드에 원하는 값 저장 가능
- 각 필드에 접근도 가능

In [6]:
p = Ether() / IP(dst="www.google.com") / TCP(flags='F')
p.summary()

'Ether / IP / TCP 141.223.140.38:ftp_data > Net("www.google.com/32"):http F'

### 패킷 캡처

> `sniff(iface, prn, filter, count, timeout) -> PacketList`

    - iface: 스니핑할 네트워크 인터페이스 (ifconfig로 확인)
    - prn: 각각의 캡처된 패킷의 콜백 함수. 콜백 함수는 캡처한 패킷을 인자로 받아야 함.
    - filter: BPF (Berkeley Packet Filter) 표현식
    - count: 캡처할 패킷 수
    - timeout: 캡처할 기간 (초) 

In [8]:
s = sniff(iface="enp0s31f6", filter="tcp", count=10)
s

<Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>

In [9]:
for packet in s:
    print(packet.summary())

Ether / IP / TCP 162.216.150.243:51873 > 141.223.122.151:28 S / Padding
Ether / IP / TCP 162.216.149.141:56971 > 141.223.140.27:4083 S / Padding
Ether / IP / TCP 185.180.143.190:26942 > 141.223.122.135:8090 S / Padding
Ether / IP / TCP 141.223.140.38:46564 > 54.95.144.31:https PA / Raw
Ether / IP / TCP 54.95.144.31:https > 141.223.140.38:46564 PA / Raw
Ether / IP / TCP 141.223.140.38:46564 > 54.95.144.31:https A
Ether / IP / TCP 141.223.140.38:48798 > 34.237.73.95:https PA / Raw
Ether / IP / TCP 34.237.73.95:https > 141.223.140.38:48798 PA / Raw
Ether / IP / TCP 141.223.140.38:48798 > 34.237.73.95:https A
Ether / IP / TCP 141.223.140.38:54478 > 34.120.208.123:https PA / Raw


In [10]:
s[0]

<Ether  dst=a4:bb:6d:a9:d6:59 src=6c:4e:f6:85:90:46 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=44 id=54321 flags= frag=0 ttl=55 proto=tcp chksum=0x6d58 src=162.216.150.243 dst=141.223.122.151 |<TCP  sport=51873 dport=28 seq=2758902701 ack=0 dataofs=6 reserved=0 flags=S window=65535 chksum=0x6307 urgptr=0 options=[('MSS', 1460)] |<Padding  load='\x00\x00' |>>>>

In [11]:
s[0].show()

###[ Ethernet ]### 
  dst       = a4:bb:6d:a9:d6:59
  src       = 6c:4e:f6:85:90:46
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 44
     id        = 54321
     flags     = 
     frag      = 0
     ttl       = 55
     proto     = tcp
     chksum    = 0x6d58
     src       = 162.216.150.243
     dst       = 141.223.122.151
     \options   \
###[ TCP ]### 
        sport     = 51873
        dport     = 28
        seq       = 2758902701
        ack       = 0
        dataofs   = 6
        reserved  = 0
        flags     = S
        window    = 65535
        chksum    = 0x6307
        urgptr    = 0
        options   = [('MSS', 1460)]
###[ Padding ]### 
           load      = '\x00\x00'



In [12]:
hexdump(s[0])

0000  A4 BB 6D A9 D6 59 6C 4E F6 85 90 46 08 00 45 00  ..m..YlN...F..E.
0010  00 2C D4 31 00 00 37 06 6D 58 A2 D8 96 F3 8D DF  .,.1..7.mX......
0020  7A 97 CA A1 00 1C A4 71 83 AD 00 00 00 00 60 02  z......q......`.
0030  FF FF 63 07 00 00 02 04 05 B4 00 00              ..c.........


In [25]:
from scapy.all import *
import sys
import socket

def select_iface():
    iface_list = socket.if_nameindex()
    
    if len(iface_list) == 0:
        print("OH NO")
        sys.exit()
        
    print("Available Network Interfaces:")
    for iface in iface_list:
        i, face = iface
        print(f"{i}: {face}")
        
    i = int(
            input(f"\nSelect network interface number to capture [1-{len(iface_list)}]: ")
        )
    return iface_list[i-1][1]

def process_packet(file_name, packet):
    wrpcap(filename= file_name, pkt = packet, append = True)
    
def main():
    file_name = str(sys.argv[1])
    duration = int(sys.argv[2])
    
    iface= select_iface()
    
    sniff(
            iface = iface,
            filter = "tcp or udp or icmp",
            prn = lambda pkt: process_packet(file_name, pkt),
            timeout=duration
    )
    
    print("DONE")
    print(f"{iface}에서 해써요! {duration}동안 캡쳐했어요! 그리고")
    print(f"{file_name}에 저장했어요~~~~")
    
    
if __name__ == "__main__":
    main()

ValueError: invalid literal for int() with base 10: '/home/piai/.local/share/jupyter/runtime/kernel-268f8a46-ed18-4c22-8eb0-45fe4e5f59b5.json'

In [18]:
from scapy.all import *
import sys
import socket

In [20]:
def select_iface():
    iface_list = socket.if_nameindex()
    
    if len(iface_list) == 0:
        print("OH NO")
        sys.exit()
        
    print("Available Network Interfaces:")
    for iface in iface_list:
        i, face = iface
        print(f"{i}: {face}")
        
    i = int(
            input(f"\nSelect network interface number to capture [1-{len(iface_list)}]: ")
        )
    return iface_list[i-1][1]

In [21]:
def process_packet(file_name, packet):
    wrpcap(filename= file_name, pkt = packet, append = True)

In [24]:
def main():
    file_name = str(sys.argv[1])
    duration = int(sys.argv[2])
    
    iface= select_iface()
    
    sniff(
            iface = iface,
            filter = "tcp or udp or icmp",
            prn = lambda pkt: process_packet(file_name, pkt),
            timeout=duration
    )
    
    print("DONE")
    print(f"{iface}에서 해써요! {duration}동안 캡쳐했어요! 그리고")
    print(f"{file_name}에 저장했어요~~~~")
    
    
if __name__ == "__main__":
    main()

ValueError: invalid literal for int() with base 10: '/home/piai/.local/share/jupyter/runtime/kernel-268f8a46-ed18-4c22-8eb0-45fe4e5f59b5.json'

### 인터페이스 리스트 출력

In [17]:
from scapy.all import *
import sys
import socket

iface_list = socket.if_nameindex()
iface_list

[(1, 'lo'), (2, 'enp2s0'), (3, 'enp0s31f6')]

### 패킷 파일 저장하기

> `wrpcap(filename, pkt)`

    - filename: 저장할 pcap, pcapng 파일 이름
    - pkt: Packet 혹은 PacketList
    - append (bool): pcap 파일에 이어서 작성하고 싶으면 True 

In [14]:
wrpcap(filename="10.pcap", pkt=s)

<img src="https://www.dropbox.com/s/bhzrdq7k6w71yvd/ex2-1.png?raw=1" width="493" height="257"/>

In [15]:
# 콜백함수 예시
def process_packet(packet):
    wrpcap(pcap_file, packet, append=True)

### 패킷 파일 불러오기

> `rdpcap(filename) -> PacketList`

    - filename: 불러올 pcap, pcapng 파일 이름 

In [16]:
pcap = rdpcap("ex1_jokyo.pcap")

FileNotFoundError: [Errno 2] No such file or directory: 'ex1_jokyo.pcap'

### 패킷 정보 확인

In [ ]:
print(f"Total number of packets: {len(pcap)}")
print(f"Bytes of packet: {len(pcap[0])}")
print(f"Packet arrival time in Unix time: {pcap[0].time}")
print(f"Packet uses UDP: {pcap[0].haslayer(UDP)}")
print(f"Src/Dst port of the packet: {pcap[0][UDP].sport}/{pcap[0][UDP].dport}")

<img src="https://www.dropbox.com/s/b25w53uvusz063u/ex2-2.png?raw=1" width="487" height="265"/>

In [ ]:
# load_contrib("gtp")
gtp = rdpcap("ex3_gtp.pcap")

In [ ]:
a = gtp[0]
a.summary()

In [ ]:
ls(GTP_U_Header())

In [ ]:
ls(a[GTP_U_Header][IP])

<img src="https://www.dropbox.com/s/00f8pqjjanl9al7/ex3.png?raw=1" width="510" height="281"/>